In [ ]:
import pandas as pd
import airportsdata
import csv
import codecs
import urllib.request
import urllib.error
import sys
import datetime as dt

cleaned_df = pd.read_csv('/Users/favea/Downloads/swiss-data/10122022b_cleaned.csv', index_col=0)

In [ ]:
airports = airportsdata.load('IATA') # key is the ICAO identifier (the default)

In [ ]:
import json

def get_data_weather(row):
    
    weather_api_endpoint = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?'
    query_params = '&contentType=json&aggregateMinutes=15&unitGroup=metric&includeAstronomy=false&collectStationContributions=false&key={}&startDateTime={}&endDateTime={}&locations={},{}'
    ApiKey='6SG4URSNKE3MLQK23A4E7R43L'
    latitude, longitude = airports[row.origin]['lat'], airports[row.origin]['lon']
    date = row.scheduled_time_departure
    start = dt.datetime.strptime(date,'%Y-%m-%d %H:%M:%S').isoformat()
    query_params=query_params.format(ApiKey, start, start,latitude,longitude)

    try:
        response = urllib.request.urlopen(weather_api_endpoint +"?"+ query_params)
        weatherData = json.loads(response.read().decode('utf-8')) 

        locations=weatherData["locations"]
        for locationid in locations:  
            location=locations[locationid]
            for value in location["values"]:


                weather = dict(origin = row.origin, destination = row.destination, scheduled_time_departure = row.scheduled_time_departure,
                               temperature = value["temp"], 
                               precipitation = value["precip"],
                               windSpeed = value["wspd"],
                               dewpoint = value["dew"],
                               humidity = value["humidity"])
                return weather
    except Exception as e:
        print(e)
        return  dict(origin = row.origin, destination = row.destination, scheduled_time_departure = row.scheduled_time_departure,
                                temperature = None, 
                               precipitation = None,
                               windSpeed =None,
                               dewpoint = None,
                               humidity = None)
        
        
    


In [ ]:
records = []
START = 3960 # Modify this (You have 1000 records free for day)
END = 4960 # modify this 

for row in cleaned_df.iloc[START:END].itertuples():
    records.append(get_data_weather(row))


In [ ]:
weather = pd.DataFrame.from_records(records)
weather = weather.drop_duplicates()

In [ ]:
out_csv = f"/Users/favea/Downloads/swiss-data/weather_{START}_{END}.csv"

In [ ]:
weather.to_csv(out_csv)

In [ ]:
import wandb

with wandb.init(
    project="swiss-delay-prediction", entity=None, job_type="weather-data"
) as run:
    table_merged = wandb.Table(dataframe=weather)

    # Create an artifact for our dataset
    dataset_artifact = wandb.Artifact(
        "dataset-weather",
        type="dataset",
        description="Table containing weather info for observations in cleaned dataframe",
    )
    # Add the table to the artifact & log the artifact
    dataset_artifact.add(table_merged, "data-table-delay-cleaned")
    dataset_artifact.add_file("/Users/favea/Downloads/swiss-data/10122022b_cleaned.csv")

    # Add the
    run.log_artifact(dataset_artifact)
